In [1]:
# Plot Grid Patches using Earth Engine
# Jupyter Notebook for visualizing the grid patches

import ee
import geemap
import geopandas as gpd
import json

# Initialize Earth Engine (make sure you're authenticated)
# ee.Authenticate()  # Run once if needed
user = "Peter"
if user == "Peter":
    ee.Initialize(project="geog761-peag224")

# %% Read the grid GeoJSON
print("Reading grid GeoJSON...")
grid = gpd.read_file('auckland_grid_100m.geojson')
print(f"Grid loaded: {len(grid)} patches")
print(f"Grid CRS: {grid.crs}")

# Convert to WGS84 for Earth Engine
grid_wgs84 = grid.to_crs('EPSG:4326')

# %% Convert grid to Earth Engine FeatureCollection
print("\nConverting grid to Earth Engine FeatureCollection...")

def gdf_to_ee_feature(row):
    """Convert a GeoDataFrame row to an ee.Feature"""
    geom = row.geometry.__geo_interface__
    properties = {
        'patch_id': int(row['patch_id']),
        'col': int(row['col']),
        'row': int(row['row'])
    }
    return ee.Feature(ee.Geometry(geom), properties)

# Convert all patches to ee.Features
features = [gdf_to_ee_feature(row) for idx, row in grid_wgs84.iterrows()]
grid_fc = ee.FeatureCollection(features)

print(f"✓ Created FeatureCollection with {len(features)} patches")

# %% Create interactive map
print("\nCreating interactive map...")
Map = geemap.Map(center=[-36.85, 174.76], zoom=10)

# Add Sentinel-2 imagery as basemap (optional)
# You can use your existing S2 image or just use a recent one
s2_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # RGB
}

# Add the grid
Map.addLayer(grid_fc, {'color': 'blue'}, 'Grid Patches', True, 0.7)

# Add grid with labels (showing patch IDs)
styled_grid = grid_fc.style(**{
    'color': 'blue',
    'fillColor': '00000000',  # Transparent fill
    'width': 2
})
Map.addLayer(styled_grid, {}, 'Grid Boundaries')

# Add layer control
Map.add_basemap('SATELLITE')

# Display the map
Map

# %% Alternative: Simple matplotlib plot
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 10))

# Plot grid
grid_wgs84.boundary.plot(ax=ax, color='blue', linewidth=1, alpha=0.7, label='Grid Patches')

# Add patch IDs for first 50 patches
subset = grid_wgs84.head(50)
for idx, row in subset.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y, str(row['patch_id']), 
            fontsize=8, ha='center', va='center', color='darkblue',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax.set_title(f'Grid Patches (showing first 50 IDs of {len(grid)} total)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% Print summary
print("\n" + "="*50)
print("GRID SUMMARY")
print("="*50)
print(f"Total patches: {len(grid)}")
print(f"Columns: {grid['col'].max() + 1}")
print(f"Rows: {grid['row'].max() + 1}")
print(f"\nBounds (WGS84):")
bounds = grid_wgs84.total_bounds
print(f"  Lon: {bounds[0]:.4f} to {bounds[2]:.4f}")
print(f"  Lat: {bounds[1]:.4f} to {bounds[3]:.4f}")
print(f"\nFirst few patches:")
print(grid.head())

# %% Export grid to Earth Engine asset (optional)
# This allows you to use it in Earth Engine directly
# Uncomment to export:
# task = ee.batch.Export.table.toAsset(
#     collection=grid_fc,
#     description='auckland_grid_5000m',
#     assetId='users/YOUR_USERNAME/auckland_grid_5000m'
# )
# task.start()
# print("Export task started. Check Tasks tab in Earth Engine Code Editor.")

EEException: Caller does not have required permission to use project geog761-peag224. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=geog761-peag224 and then retry. Propagation of the new permission may take a few minutes.